## Google Drive Auth

In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# import getpass

# auth.authenticate_user()
# creds = GoogleCredentials.get_application_default()
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

## Google drive path mapping

In [0]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [3]:
!ls

sample_data


In [4]:
!rm -rf data
!ls

sample_data


In [5]:
!echo --------Downloading img.zip--------
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7EVK8r0v71pa2EyNEJ0dE9zbU0' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B7EVK8r0v71pa2EyNEJ0dE9zbU0" -O deepfashion_img.zip
!rm -rf /tmp/cookies.txt

!echo --------Data download--------
!ls

--------Downloading img.zip--------

Redirecting output to ‘wget-log’.

Redirecting output to ‘wget-log.1’.
--------Data download--------
deepfashion_img.zip  sample_data  wget-log  wget-log.1


In [6]:
!echo --------Downloading list_eval_partition.txt--------
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7EVK8r0v71pdS1FMlNreEwtc1E' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B7EVK8r0v71pdS1FMlNreEwtc1E" -O list_eval_partition.txt
!rm -rf /tmp/cookies.txt

!echo --------Data download--------
!ls

--------Downloading list_eval_partition.txt--------

Redirecting output to ‘wget-log.2’.

Redirecting output to ‘wget-log.3’.
--------Data download--------
deepfashion_img.zip	 sample_data  wget-log.1  wget-log.3
list_eval_partition.txt  wget-log     wget-log.2


In [7]:

!echo --------Downloading Anno.zip--------
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-zjMCY-YERKhA9OmV78j-s678n_qAqPb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-zjMCY-YERKhA9OmV78j-s678n_qAqPb" -O Anno.zip
!rm -rf /tmp/cookies.txt

!echo --------Data download--------
!ls

--------Downloading Anno.zip--------

Redirecting output to ‘wget-log.4’.

Redirecting output to ‘wget-log.5’.
--------Data download--------
Anno.zip		 sample_data  wget-log.2  wget-log.5
deepfashion_img.zip	 wget-log     wget-log.3
list_eval_partition.txt  wget-log.1   wget-log.4


In [8]:
!echo --------Creating paths---------
!mkdir data
!mkdir data/val
!mkdir data/Anno
!mkdir data/Eval

!echo --------ls into data/---------
!ls data

--------Creating paths---------
--------ls into data/---------
Anno  Eval  val


In [9]:
!echo --------moving the .zip files---------
!mv ./deepfashion_img.zip ./data/
!mv list_eval_partition.txt ./data/Eval/
!mv ./Anno.zip ./data/Anno/

!echo --------moved Data--------
!ls

--------moving the .zip files---------
--------moved Data--------
data	     wget-log	 wget-log.2  wget-log.4
sample_data  wget-log.1  wget-log.3  wget-log.5


In [10]:
!echo --------installing unzip---------
!apt-get install unzip

--------installing unzip---------
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [11]:
!ls data/

Anno  deepfashion_img.zip  Eval  val


In [12]:
!echo --------unziping---------
!unzip -qq ./data/deepfashion_img.zip -d ./data/
!unzip -qq ./data/Anno/Anno.zip -d ./data/Anno/

!ls ./data/img/ -U | head -4

--------unziping---------
Ornate_Embroidered-Mesh_Dress
Drawstring_Denim_Cargo_Joggers
Boxy_Tank_Top
Bold_Baja_Knit_Shorts


In [13]:
!echo --------romoving .zip files---------
!rm -rf ./data/deepfashion_img.zip
!rm -rf ./data/Anno/Anno.zip

--------romoving .zip files---------


In [14]:
!ls ./data/img/ -U | head -4

Ornate_Embroidered-Mesh_Dress
Drawstring_Denim_Cargo_Joggers
Boxy_Tank_Top
Bold_Baja_Knit_Shorts


## Let's start!

In [0]:
import shutil #file manager
import os
import re #regular exprestion
import cv2
# will use them for creating custom directory iterator
import numpy as np
from six.moves import range
# regular expression for splitting by whitespace
splitter = re.compile("\s+")
base_path = './data/'

## Prepare the files
Run it on your local machine and upload the files to google drive

In [0]:
def process_folders():
#   TODO: fix this bad code
    base_path = './data/img'
    # Read the relevant annotation file and preprocess it
    # Assumed that the annotation files are under '<project folder>/data/anno' path
    with open('./data/Eval/list_eval_partition.txt', 'r') as eval_partition_file:
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_all = [(v[0][4:], v[0].split('/')[1].split('_')[-1], v[1]) for v in list_eval_partition]
    
    
    # Put each image into the relevant folder in train/test/validation folder
    for element in list_all:
        if not os.path.exists(os.path.join(base_path, element[2])):
            os.mkdir(os.path.join(base_path, element[2]))
        if not os.path.exists(os.path.join(os.path.join(base_path, element[2]), element[1])):
            os.mkdir(os.path.join(os.path.join(base_path, element[2]), element[1]))
        if not os.path.exists(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                              element[0].split('/')[0])):
            os.mkdir(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                     element[0].split('/')[0]))
        shutil.move(os.path.join(base_path, element[0]),
                    os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1]), element[0]))
    base_path = './data/'

In [0]:
process_folders()

In [18]:
!echo ---------ls-------------
!ls
!echo ---------ls data-------------
!ls data
!echo ---------ls data/img max 4 --------------
!ls ./data/img/ -U | head -4 
!echo ---------ls data/img/train max 4 --------------
!ls ./data/img/train -U | head -4

---------ls-------------
data	     wget-log	 wget-log.2  wget-log.4
sample_data  wget-log.1  wget-log.3  wget-log.5
---------ls data-------------
Anno  Eval  img  val
---------ls data/img max 4 --------------
Ornate_Embroidered-Mesh_Dress
Drawstring_Denim_Cargo_Joggers
Boxy_Tank_Top
Bold_Baja_Knit_Shorts
---------ls data/img/train max 4 --------------
Turtleneck
Sarong
Jodhpurs
Tee


In [0]:
def create_dict_bboxes(list_all, split='train'):
    lst = [(line[0], line[1], line[3], line[2]) for line in list_all if line[2] == split]
    lst = [("".join(line[0].split('/')[0] + '/' + line[3] + '/' + line[1] + line[0][3:]), line[1], line[2]) for line in lst]
    lst_shape = [cv2.imread('./data/' + line[0]).shape for line in lst]
    lst = [(line[0], line[1], (round(line[2][0] / shape[1], 2), round(line[2][1] / shape[0], 2), round(line[2][2] / shape[1], 2), round(line[2][3] / shape[0], 2))) for line, shape in zip(lst, lst_shape)]
    dict_ = {}
    dones = 0
    for line, shape in zip(lst, lst_shape):
      try:
        dict_["/".join(line[0].split('/')[2:])] = {'x1': line[2][0], 'y1': line[2][1], 'x2': line[2][2], 'y2': line[2][3], 'shape': shape}
        dones = dones + 1
      except:
        print('EXCEPTION: ' + str(shape))
        break
    print('Dones: ' + str(dones))
#       dict_ = {"/".join(line[0].split('/')[2:]): {'x1': line[2][0], 'y1': line[2][1], 'x2': line[2][2], 'y2': line[2][3], 'shape': line[2][4]} for line in lst}
    return dict_

In [0]:
def get_dict_bboxes():
    i = 0
    with open(base_path + 'Anno/list_category_img.txt', 'r') as category_img_file, \
         open(base_path + 'Anno/list_eval_partition.txt', 'r') as eval_partition_file, \
         open(base_path + 'Anno/list_bbox.txt', 'r') as bbox_file:
        list_category_img = [line.rstrip('\n') for line in category_img_file][2:]
        list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
        list_bbox = [line.rstrip('\n') for line in bbox_file][2:]

        list_category_img = [splitter.split(line) for line in list_category_img]
        list_eval_partition = [splitter.split(line) for line in list_eval_partition]
        list_bbox = [splitter.split(line) for line in list_bbox]

        list_all = [(k[0], k[0].split('/')[1].split('_')[-1], v[1], (int(b[1]), int(b[2]), int(b[3]), int(b[4])))
                    for k, v, b in zip(list_category_img, list_eval_partition, list_bbox)]

        list_all.sort(key=lambda x: x[1])
        dict_train = create_dict_bboxes(list_all)
#         print('1')
        dict_val = create_dict_bboxes(list_all, split='val')
#         print('2')
        dict_test = create_dict_bboxes(list_all, split='test')
#         print('3')

        return dict_train, dict_val, dict_test

In [21]:
from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras import backend as K

Using TensorFlow backend.


In [22]:
model_resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

94658560/94653016 [==============================] - 13s 0us/step


In [0]:
for layer in model_resnet.layers[:-12]:
    # 6 - 12 - 18 have been tried. 12 is the best.
    layer.trainable = False

In [0]:
x = model_resnet.output
x = Dense(512, activation='elu', kernel_regularizer=l2(0.001))(x)
y = Dense(46, activation='softmax', name='img')(x)

In [0]:
x_bbox = model_resnet.output
x_bbox = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
x_bbox = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x_bbox)
bbox = Dense(4, kernel_initializer='normal', name='bbox')(x_bbox)

In [0]:
final_model = Model(inputs=model_resnet.input,
                    outputs=[y, bbox])

In [27]:
print(final_model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
opt = SGD(lr=0.0001, momentum=0.9, nesterov=True)

In [0]:
final_model.compile(optimizer=opt,
                    loss={'img': 'categorical_crossentropy',
                          'bbox': 'mean_squared_error'},
                    metrics={'img': ['accuracy', 'top_k_categorical_accuracy'], # default: top-5
                             'bbox': ['mse']})

In [0]:
train_datagen = ImageDataGenerator(rotation_range=30.,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator()

In [0]:
class DirectoryIteratorWithBoundingBoxes(DirectoryIterator):
    def __init__(self, directory, image_data_generator, bounding_boxes: dict = None, target_size=(256, 256),
                 color_mode: str = 'rgb', classes=None, class_mode: str = 'categorical', batch_size: int = 32,
                 shuffle: bool = True, seed=None, data_format=None, save_to_dir=None,
                 save_prefix: str = '', save_format: str = 'jpeg', follow_links: bool = False):
        super().__init__(directory, image_data_generator, target_size, color_mode, classes, class_mode, batch_size,
                         shuffle, seed, data_format, save_to_dir, save_prefix, save_format, follow_links)
        self.bounding_boxes = bounding_boxes

    def next(self):
        """
        # Returns
            The next batch.
        """
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        locations = np.zeros((len(batch_x),) + (4,), dtype=K.floatx())

        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img = image.load_img(os.path.join(self.directory, fname),
                                 grayscale=grayscale,
                                 target_size=self.target_size)
            x = image.img_to_array(img, data_format=self.data_format)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x

            if self.bounding_boxes is not None:
                bounding_box = self.bounding_boxes[fname]
                locations[i] = np.asarray(
                    [bounding_box['x1'], bounding_box['y1'], bounding_box['x2'], bounding_box['y2']],
                    dtype=K.floatx())
        # optionally save augmented images to disk for debugging purposes
        # build batch of labels
        if self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), 46), dtype=K.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x

        if self.bounding_boxes is not None:
            return batch_x, [batch_y, locations]
        else:
            return batch_x, batch_y

In [32]:
!ls ./data/img/train/Romper/Baroque_Embroidered_Romper

img_00000001.jpg  img_00000016.jpg  img_00000028.jpg  img_00000042.jpg
img_00000002.jpg  img_00000017.jpg  img_00000029.jpg  img_00000043.jpg
img_00000006.jpg  img_00000019.jpg  img_00000030.jpg  img_00000045.jpg
img_00000007.jpg  img_00000020.jpg  img_00000031.jpg  img_00000046.jpg
img_00000008.jpg  img_00000022.jpg  img_00000032.jpg  img_00000049.jpg
img_00000009.jpg  img_00000023.jpg  img_00000033.jpg  img_00000050.jpg
img_00000010.jpg  img_00000024.jpg  img_00000035.jpg  img_00000051.jpg
img_00000011.jpg  img_00000025.jpg  img_00000037.jpg  img_00000052.jpg
img_00000012.jpg  img_00000026.jpg  img_00000039.jpg
img_00000015.jpg  img_00000027.jpg  img_00000041.jpg


In [33]:
dict_train, dict_val, dict_test = get_dict_bboxes()

Dones: 209222
Dones: 40000
Dones: 40000


In [34]:
print(dict_val)
print(type(dict_val))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
train_iterator = DirectoryIteratorWithBoundingBoxes(base_path + 'img/train', train_datagen, bounding_boxes=dict_train, target_size=(200, 200))
test_iterator = DirectoryIteratorWithBoundingBoxes(base_path + 'img/val', test_datagen, bounding_boxes=dict_val,target_size=(200, 200))

Found 209222 images belonging to 46 classes.
Found 40000 images belonging to 46 classes.


In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               patience=12,
                               factor=0.5,
                               verbose=1)
tensorboard = TensorBoard(log_dir='./logs')
early_stopper = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=1)
checkpoint = ModelCheckpoint('./models/model.h5')

In [0]:
def custom_generator(iterator):
  while True:
    batch_x, batch_y = iterator.next()
    yield(batch_x,batch_y)

In [0]:
!mkdir models
!touch models/model.h5

In [39]:
!ls

data	sample_data  wget-log.1  wget-log.3  wget-log.5
models	wget-log     wget-log.2  wget-log.4


In [40]:
def train(epochs, validation_steps):
  final_model.fit_generator(custom_generator(train_iterator),
                            steps_per_epoch=2000,
                            epochs=epochs, validation_data=custom_generator(test_iterator),
                            validation_steps=validation_steps,
                            verbose=2,
                            shuffle=True,
                            callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                            workers=12)

Epoch 1/5
 - 881s - loss: 4.3517 - img_loss: 2.4676 - bbox_loss: 0.0451 - img_acc: 0.3345 - img_top_k_categorical_accuracy: 0.6620 - bbox_mean_squared_error: 0.0451 - val_loss: 4.2792 - val_img_loss: 2.3799 - val_bbox_loss: 0.0677 - val_img_acc: 0.3625 - val_img_top_k_categorical_accuracy: 0.7188 - val_bbox_mean_squared_error: 0.0677
Epoch 2/5
 - 885s - loss: 3.9373 - img_loss: 2.0843 - bbox_loss: 0.0286 - img_acc: 0.4147 - img_top_k_categorical_accuracy: 0.7607 - bbox_mean_squared_error: 0.0286 - val_loss: 3.8302 - val_img_loss: 1.9461 - val_bbox_loss: 0.0668 - val_img_acc: 0.4313 - val_img_top_k_categorical_accuracy: 0.8187 - val_bbox_mean_squared_error: 0.0668
Epoch 3/5
 - 895s - loss: 3.7821 - img_loss: 1.9474 - bbox_loss: 0.0247 - img_acc: 0.4454 - img_top_k_categorical_accuracy: 0.7934 - bbox_mean_squared_error: 0.0247 - val_loss: 3.9309 - val_img_loss: 2.0728 - val_bbox_loss: 0.0552 - val_img_acc: 0.4188 - val_img_top_k_categorical_accuracy: 0.8187 - val_bbox_mean_squared_error:

In [44]:
final_model.fit_generator(custom_generator(train_iterator),
                          steps_per_epoch=2000,
                          epochs=5, validation_data=custom_generator(test_iterator),
                          validation_steps=5,
                          verbose=2,
                          shuffle=True,
                          callbacks=[lr_reducer, checkpoint, early_stopper, tensorboard],
                          workers=12)

Epoch 1/5
 - 893s - loss: 3.5693 - img_loss: 1.7820 - bbox_loss: 0.0199 - img_acc: 0.4853 - img_top_k_categorical_accuracy: 0.8244 - bbox_mean_squared_error: 0.0199 - val_loss: 3.6803 - val_img_loss: 1.8814 - val_bbox_loss: 0.0385 - val_img_acc: 0.4625 - val_img_top_k_categorical_accuracy: 0.8313 - val_bbox_mean_squared_error: 0.0385
Epoch 2/5
 - 881s - loss: 3.5222 - img_loss: 1.7498 - bbox_loss: 0.0190 - img_acc: 0.4917 - img_top_k_categorical_accuracy: 0.8308 - bbox_mean_squared_error: 0.0190 - val_loss: 3.8882 - val_img_loss: 2.1112 - val_bbox_loss: 0.0305 - val_img_acc: 0.4813 - val_img_top_k_categorical_accuracy: 0.7875 - val_bbox_mean_squared_error: 0.0305
Epoch 3/5
 - 881s - loss: 3.4711 - img_loss: 1.7130 - bbox_loss: 0.0185 - img_acc: 0.5024 - img_top_k_categorical_accuracy: 0.8380 - bbox_mean_squared_error: 0.0185 - val_loss: 3.7174 - val_img_loss: 1.9545 - val_bbox_loss: 0.0302 - val_img_acc: 0.4250 - val_img_top_k_categorical_accuracy: 0.7875 - val_bbox_mean_squared_error:

In [41]:
!ls models

model.h5


In [42]:
!ls data/img/train/Joggers/Zippered_Camo-Paneled_Chino_Joggers

img_00000001.jpg  img_00000006.jpg  img_00000012.jpg  img_00000018.jpg
img_00000002.jpg  img_00000007.jpg  img_00000013.jpg  img_00000019.jpg
img_00000003.jpg  img_00000008.jpg  img_00000014.jpg  img_00000021.jpg
img_00000004.jpg  img_00000009.jpg  img_00000015.jpg  img_00000022.jpg
img_00000005.jpg  img_00000011.jpg  img_00000017.jpg


In [43]:
test_datagen = ImageDataGenerator()

test_iterator = DirectoryIteratorWithBoundingBoxes(base_path +"img/test", test_datagen, bounding_boxes=dict_test, target_size=(200, 200))
scores = final_model.evaluate_generator(test_iterator.next(), steps=2000)

print('Multi target loss: ' + str(scores[0]))
print('Image loss: ' + str(scores[1]))
print('Bounding boxes loss: ' + str(scores[2]))
print('Image accuracy: ' + str(scores[3]))
print('Top-5 image accuracy: ' + str(scores[4]))
print('Bounding boxes error: ' + str(scores[5]))

Found 40000 images belonging to 46 classes.


TypeError: ignored